In [ ]:
!pip install influxdb-client

In [ ]:
import os
import time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# Configurar o token (substitua pelo seu token real)
os.environ["INFLUXDB_TOKEN"] = "4jCdtbcB3entE_0cy7RJ9ucL1dfBUVcV9pyvIFcThEfocGd1mbJpvNMWzxYdRa9Z0rUu2toYzSkb0c-jdRigcw=="

token = os.environ.get("INFLUXDB_TOKEN")
org = "feira"
url = "https://tsdb.feira-de-jogos.dev.br"

# Criar o cliente
client = InfluxDBClient(url=url, token=token, org=org)

In [ ]:
buckets_api = client.buckets_api()
buckets = buckets_api.find_buckets().buckets

print("📊 Buckets disponíveis:")
for i, bucket in enumerate(buckets, 1):
    print(f"{i}. {bucket.name}")

if buckets:
    print(f"\n💡 Sugestão: Use o bucket '{buckets[0].name}' no seu código!")
else:
    print("\n❌ Nenhum bucket encontrado. Você precisa criar um primeiro.")


📊 Buckets disponíveis:
1. _monitoring
2. feira
3. _tasks

💡 Sugestão: Use o bucket '_monitoring' no seu código!


In [ ]:
print("\n📊 Tentando ler os dados...")
query_api = client.query_api()

try:
    # Consulta mais simples primeiro
    query = f'from(bucket: "feira") |> range(start: -1h)'
    tables = query_api.query(query, org=org)

    print("✅ Conexão bem-sucedida! Dados encontrados:")
    record_count = 0
    for table in tables:
        for record in table.records:
            print(f"Medição: {record.get_measurement()}, Campo: {record.get_field()}, Valor: {record.get_value()}")
            record_count += 1

    if record_count == 0:
        print("ℹ️  Nenhum dado encontrado no período. Vamos tentar uma consulta mais específica...")

        # Consulta específica para nossos dados de teste
        query_specific = f'from(bucket: "feira") |> range(start: -1h) |> filter(fn: (r) => r._measurement == "test_measurement")'
        tables_specific = query_api.query(query_specific, org=org)

        specific_count = 0
        for table in tables_specific:
            for record in table.records:
                print(f"📈 Nosso dado: Tempo: {record.get_time()}, Valor: {record.get_value()}")
                specific_count += 1

        if specific_count == 0:
            print("❌ Nossos dados de teste não foram encontrados. Pode haver problema com a escrita.")

except Exception as e:
    print(f"❌ Erro na leitura: {e}")



📊 Tentando ler os dados...
✅ Conexão bem-sucedida! Dados encontrados:
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.747
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.695
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.745
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.768
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.792
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.748
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.816
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.756
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.799
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bmp280_press, Valor: 1019.731
Medição: em/fa875d3f-d1ef-4c27-b774-b41c69d70608, Campo: bm

In [ ]:
# Calcular médias de cada sensor
query_avg = '''
from(bucket: "feira")
  |> range(start: -1h)
  |> filter(fn: (r) => r._measurement == "em/fa875d3f-d1ef-4c27-b774-b41c69d70608")
  |> mean()
'''

tables_avg = query_api.query(query_avg, org=org)

print("📊 Médias dos últimos 60 minutos:")
for table in tables_avg:
    for record in table.records:
        print(f"{record.get_field()}: {record.get_value():.2f}")

📊 Médias dos últimos 60 minutos:
bmp280_press: 1019.57
bmp280_temp: 19.80
dht11_temp: 18.18
dht11_umid: 61.09
ds18b20_temp: 19.46
mq4_ch4: 18.68
mq7_co: 1.13
sensor_chuva: 1804.25


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Converter dados para DataFrame
data = []
for table in tables:
    for record in table.records:
        data.append({
            'time': record.get_time(),
            'sensor': record.get_field(),
            'value': record.get_value()
        })

df = pd.DataFrame(data)
print(df.head())

                       time        sensor     value
0 2025-09-26 13:04:58+00:00  bmp280_press  1019.747
1 2025-09-26 13:06:00+00:00  bmp280_press  1019.695
2 2025-09-26 13:07:02+00:00  bmp280_press  1019.745
3 2025-09-26 13:08:03+00:00  bmp280_press  1019.768
4 2025-09-26 13:09:05+00:00  bmp280_press  1019.792
